## Setup and Imports

In [1]:
import warnings
warnings.filterwarnings("ignore")
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

!pip install kagglehub timm torchmetrics pytorch_lightning trackio -qq

import numpy as np
import pandas as pd
import torch
from torch import nn
from torch.utils.data import DataLoader, Dataset
import torch.nn.functional as F
from PIL import Image
from pathlib import Path
import timm
from torchvision import transforms
from tqdm import tqdm
import pytorch_lightning as pl
import torchmetrics
from sklearn.model_selection import train_test_split
import kagglehub
import trackio
import json
from datetime import datetime
from pytorch_lightning.callbacks import ModelCheckpoint, EarlyStopping, Callback

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.6/68.6 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 875.0/875.0 kB 17.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.5/63.5 MB 28.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 325.4/325.4 kB 24.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 104.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 84.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 65.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 40.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 27.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

## Configuration and TrackIO Setup

In [2]:
class Config:
    SEED = 42
    IMG_SIZE = 384
    BATCH_SIZE = 16
    NUM_AGE_BINS = 101
    EPOCHS = 5
    DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    LEARNING_RATE = 2e-4
    
    TRAIN_CSV = "/kaggle/input/sep-25-dl-gen-ai-nppe-1/face_dataset/train.csv"
    TEST_CSV = "/kaggle/input/sep-25-dl-gen-ai-nppe-1/face_dataset/test.csv"
    ROOT_DIR = "/kaggle/input/sep-25-dl-gen-ai-nppe-1/face_dataset"

torch.manual_seed(Config.SEED)
np.random.seed(Config.SEED)

print(f"Device: {Config.DEVICE}")

from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
os.environ['HF_TOKEN'] = user_secrets.get_secret("NPPE1")

run = trackio.init(
    project="25-t3-nppe1",
    name=f"EB4-{datetime.now().strftime('%Y-%m-%d-%H-%M-%S')}",
    group="efficientnet-b4-training",
    space_id="Abhishek-A0/dlgenai-nppe"
)

trackio.log({
    "seed": Config.SEED,
    "img_size": Config.IMG_SIZE,
    "batch_size": Config.BATCH_SIZE,
    "num_age_bins": Config.NUM_AGE_BINS,
    "epochs": Config.EPOCHS,
    "device": str(Config.DEVICE),
    "model_backbone": "tf_efficientnet_b4_ns",
    "learning_rate": Config.LEARNING_RATE
})

Device: cuda
* Trackio project initialized: 25-t3-nppe1
* Trackio metrics will be synced to Hugging Face Dataset: Abhishek-A0/dlgenai-nppe-dataset
* Found existing space: https://huggingface.co/spaces/Abhishek-A0/dlgenai-nppe
* View dashboard by going to: https://Abhishek-A0-dlgenai-nppe.hf.space/


* Created new run: EB4-2025-11-12-12-18-49


In [3]:
# class FocalLoss(nn.Module):
#     def __init__(self, alpha=0.5, gamma=2.0):
#         super().__init__()
#         self.alpha = alpha
#         self.gamma = gamma
        
#     def forward(self, inputs, targets):
#         BCE_loss = F.cross_entropy(inputs, targets, reduction='none')
#         pt = torch.exp(-BCE_loss)
#         focal_loss = self.alpha * (1-pt)**self.gamma * BCE_loss
#         return focal_loss.mean()

# class FixedFaceAgeGenderModel(pl.LightningModule):  
#     def __init__(self, backbone_name="tf_efficientnet_b4_ns", num_age_bins=101, lr=2e-4):
#         super().__init__()
#         self.save_hyperparameters()
        
#         # Backbone with proper global pooling
#         self.backbone = timm.create_model(
#             backbone_name, 
#             pretrained=True,
#             num_classes=0,
#             global_pool='avg'
#         )
#         feat_dim = self.backbone.num_features
        
#         print(f"📊 Backbone feature dimension: {feat_dim}")
        
#         self.attention = nn.Sequential(
#             nn.Linear(feat_dim, 256),
#             nn.ReLU(),
#             nn.Dropout(0.2),
#             nn.Linear(256, feat_dim),
#             nn.Sigmoid()
#         )
        
#         self.gender_head = nn.Sequential(
#             nn.Linear(feat_dim, 256),
#             nn.BatchNorm1d(256),
#             nn.ReLU(),
#             nn.Dropout(0.3),
#             nn.Linear(256, 128),
#             nn.ReLU(),
#             nn.Dropout(0.2),
#             nn.Linear(128, 2)
#         )
        
#         self.age_head = nn.Sequential(
#             nn.Linear(feat_dim, 256),
#             nn.BatchNorm1d(256),
#             nn.ReLU(),
#             nn.Dropout(0.3),
#             nn.Linear(256, 128),
#             nn.ReLU(),
#             nn.Dropout(0.2),
#             nn.Linear(128, num_age_bins)
#         )
        
#         self.gender_loss_fn = nn.CrossEntropyLoss()
#         self.age_loss_fn = nn.KLDivLoss(reduction='batchmean')
        
#         self.train_gender_acc = torchmetrics.classification.Accuracy(task="multiclass", num_classes=2)
#         self.val_gender_acc = torchmetrics.classification.Accuracy(task="multiclass", num_classes=2)
        
#         self.age_bins = torch.arange(0, num_age_bins).float()

#     def forward(self, x):
#         features = self.backbone(x)
#         attention_weights = self.attention(features)
#         attended_features = features * attention_weights
        
#         gender_logits = self.gender_head(attended_features)
#         age_logits = self.age_head(attended_features)
        
#         return gender_logits, age_logits

#     def configure_optimizers(self):
#         optimizer = torch.optim.AdamW(
#             self.parameters(), 
#             lr=self.hparams.lr,
#             weight_decay=0.01,
#         )
        
#         scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(
#             optimizer, 
#             T_max=Config.EPOCHS  
#         )
        
#         return [optimizer], [scheduler]

#     def _shared_step(self, batch, stage):
#         imgs, genders, age_targets = batch
#         gender_logits, age_logits = self(imgs)
        
#         gender_loss = self.gender_loss_fn(gender_logits, genders)
#         age_loss = self.age_loss_fn(F.log_softmax(age_logits, dim=1), age_targets)
#         total_loss = gender_loss + age_loss
        
#         # Gender metrics
#         if stage == 'train':
#             self.train_gender_acc(gender_logits, genders)
#             gender_acc = self.train_gender_acc.compute()
#         else:
#             self.val_gender_acc(gender_logits, genders)
#             gender_acc = self.val_gender_acc.compute()
        
#         # Age metrics
#         age_probs = F.softmax(age_logits, dim=1)
#         pred_ages = (age_probs * self.age_bins.to(age_probs.device)).sum(dim=1)
#         true_ages = (age_targets * self.age_bins.to(age_targets.device)).sum(dim=1)
#         age_rmse = torch.sqrt(F.mse_loss(pred_ages, true_ages))
#         age_mae = F.l1_loss(pred_ages, true_ages)
        
#         # Log metrics to PyTorch Lightning
#         self.log(f"{stage}_loss", total_loss, prog_bar=True)
#         self.log(f"{stage}_gender_loss", gender_loss, prog_bar=True)
#         self.log(f"{stage}_age_loss", age_loss, prog_bar=True)
#         self.log(f"{stage}_gender_acc", gender_acc, prog_bar=True)
#         self.log(f"{stage}_age_rmse", age_rmse, prog_bar=True)
#         self.log(f"{stage}_age_mae", age_mae, prog_bar=True)
        
#         return total_loss

#     def training_step(self, batch, batch_idx):
#         return self._shared_step(batch, "train")

#     def validation_step(self, batch, batch_idx):
#         self._shared_step(batch, "val")

## Model Architecture

### FocalLoss + EfficientNet Model

In [4]:
class FocalLoss(nn.Module):
    def __init__(self, alpha=0.5, gamma=2.0):
        super().__init__()
        self.alpha = alpha
        self.gamma = gamma
        
    def forward(self, inputs, targets):
        BCE_loss = F.cross_entropy(inputs, targets, reduction='none')
        pt = torch.exp(-BCE_loss)
        focal_loss = self.alpha * (1-pt)**self.gamma * BCE_loss
        return focal_loss.mean()

class FixedFaceAgeGenderModel(pl.LightningModule):
    def __init__(self, backbone_name="tf_efficientnet_b4_ns", num_age_bins=101, lr=2e-4):
        super().__init__()
        self.save_hyperparameters()
        
        # ✅ PROVEN ARCHITECTURE - Your 0.84 Winner
        self.backbone = timm.create_model(
            backbone_name, 
            pretrained=True,
            num_classes=0,
            global_pool='avg'  # ✅ Your winning setting
        )
        feat_dim = self.backbone.num_features
        
        print(f"📊 Backbone feature dimension: {feat_dim}")
        
        # ✅ PROVEN ATTENTION MECHANISM
        self.attention = nn.Sequential(
            nn.Linear(feat_dim, 256),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(256, feat_dim),
            nn.Sigmoid()
        )
        
        # ✅ PROVEN GENDER HEAD
        self.gender_head = nn.Sequential(
            nn.Linear(feat_dim, 256),
            nn.BatchNorm1d(256),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(256, 128),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(128, 2)
        )
        
        # ✅ PROVEN AGE HEAD  
        self.age_head = nn.Sequential(
            nn.Linear(feat_dim, 256),
            nn.BatchNorm1d(256),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(256, 128),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(128, num_age_bins)
        )
        
        # ✅ PROVEN LOSS FUNCTIONS
        self.gender_loss_fn = nn.CrossEntropyLoss()
        self.age_loss_fn = nn.KLDivLoss(reduction='batchmean')
        
        # ✅ PROVEN METRICS
        self.train_gender_acc = torchmetrics.classification.Accuracy(task="multiclass", num_classes=2)
        self.val_gender_acc = torchmetrics.classification.Accuracy(task="multiclass", num_classes=2)
        
        self.age_bins = torch.arange(0, num_age_bins).float()

    def forward(self, x):
        # ✅ PROVEN FORWARD PASS
        features = self.backbone(x)
        attention_weights = self.attention(features)
        attended_features = features * attention_weights
        
        gender_logits = self.gender_head(attended_features)
        age_logits = self.age_head(attended_features)
        
        return gender_logits, age_logits

    def configure_optimizers(self):
        # ✅ PROVEN OPTIMIZER SETTINGS
        optimizer = torch.optim.AdamW(
            self.parameters(), 
            lr=self.hparams.lr,
            weight_decay=0.01,  # ✅ Your winning regularization
        )
        
        scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(
            optimizer, 
            T_max=Config.EPOCHS
        )
        
        return [optimizer], [scheduler]

    def _shared_step(self, batch, stage):
        imgs, genders, age_targets = batch
        gender_logits, age_logits = self(imgs)
        
        gender_loss = self.gender_loss_fn(gender_logits, genders)
        age_loss = self.age_loss_fn(F.log_softmax(age_logits, dim=1), age_targets)
        total_loss = gender_loss + age_loss
        
        # ✅ PROVEN LOGGING STRATEGY
        self.log(f"{stage}_loss", total_loss, prog_bar=True)
        self.log(f"{stage}_gender_loss", gender_loss, prog_bar=True)
        self.log(f"{stage}_age_loss", age_loss, prog_bar=True)
        
        # ✅ PROVEN METRICS TRACKING
        if stage == 'train':
            self.train_gender_acc(gender_logits, genders)
            self.log(f"{stage}_gender_acc", self.train_gender_acc, on_step=True, on_epoch=True, prog_bar=True)
        else:
            self.val_gender_acc(gender_logits, genders)
            self.log(f"{stage}_gender_acc", self.val_gender_acc, on_epoch=True, prog_bar=True)
        
        # Age metrics
        age_probs = F.softmax(age_logits, dim=1)
        pred_ages = (age_probs * self.age_bins.to(age_probs.device)).sum(dim=1)
        true_ages = (age_targets * self.age_bins.to(age_targets.device)).sum(dim=1)
        age_rmse = torch.sqrt(F.mse_loss(pred_ages, true_ages))
        age_mae = F.l1_loss(pred_ages, true_ages)
        
        self.log(f"{stage}_age_rmse", age_rmse, prog_bar=True)
        self.log(f"{stage}_age_mae", age_mae, prog_bar=True)
        
        return total_loss

    def training_step(self, batch, batch_idx):
        return self._shared_step(batch, "train")

    def validation_step(self, batch, batch_idx):
        self._shared_step(batch, "val")


###  Simple CNN Model 

In [5]:
class SimpleCNN(nn.Module):
    def __init__(self, num_age_bins=101):
        super().__init__()
        
        self.features = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=3, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True),
            nn.Conv2d(64, 64, kernel_size=3, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Dropout2d(0.25),
            
            nn.Conv2d(64, 128, kernel_size=3, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(inplace=True),
            nn.Conv2d(128, 128, kernel_size=3, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Dropout2d(0.25),
            
            nn.Conv2d(128, 256, kernel_size=3, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(inplace=True),
            nn.Conv2d(256, 256, kernel_size=3, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Dropout2d(0.25),
            
            nn.Conv2d(256, 512, kernel_size=3, padding=1),
            nn.BatchNorm2d(512),
            nn.ReLU(inplace=True),
            nn.Conv2d(512, 512, kernel_size=3, padding=1),
            nn.BatchNorm2d(512),
            nn.ReLU(inplace=True),
            nn.AdaptiveAvgPool2d((1, 1)),
            nn.Dropout2d(0.25),
        )
        
        self.gender_classifier = nn.Sequential(
            nn.Dropout(0.5),
            nn.Linear(512, 256),
            nn.ReLU(inplace=True),
            nn.Dropout(0.5),
            nn.Linear(256, 2)
        )
        
        self.age_classifier = nn.Sequential(
            nn.Dropout(0.5),
            nn.Linear(512, 256),
            nn.ReLU(inplace=True),
            nn.Dropout(0.5),
            nn.Linear(256, num_age_bins)
        )
        
    def forward(self, x):
        features = self.features(x)
        features = features.view(features.size(0), -1)
        
        gender_out = self.gender_classifier(features)
        age_out = self.age_classifier(features)
        
        return gender_out, age_out

class SimpleFaceAgeGenderModel(pl.LightningModule):
    def __init__(self, num_age_bins=101, lr=1e-3):
        super().__init__()
        self.save_hyperparameters()
        
        self.cnn = SimpleCNN(num_age_bins)
        
        self.gender_loss_fn = nn.CrossEntropyLoss()
        self.age_loss_fn = nn.KLDivLoss(reduction='batchmean')
        
        self.train_gender_acc = torchmetrics.classification.Accuracy(task="multiclass", num_classes=2)
        self.val_gender_acc = torchmetrics.classification.Accuracy(task="multiclass", num_classes=2)
        
        self.age_bins = torch.arange(0, num_age_bins).float()

    def forward(self, x):
        return self.cnn(x)

    def configure_optimizers(self):
        optimizer = torch.optim.AdamW(
            self.parameters(), 
            lr=self.hparams.lr,
            weight_decay=1e-4,
        )
        
        scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(
            optimizer, 
            T_max=Config.EPOCHS
        )
        
        return [optimizer], [scheduler]

    def _shared_step(self, batch, stage):
        imgs, genders, age_targets = batch
        gender_logits, age_logits = self(imgs)
        
        gender_loss = self.gender_loss_fn(gender_logits, genders)
        age_loss = self.age_loss_fn(F.log_softmax(age_logits, dim=1), age_targets)
        total_loss = gender_loss + age_loss
        
        if stage == 'train':
            self.train_gender_acc(gender_logits, genders)
            gender_acc = self.train_gender_acc.compute()
        else:
            self.val_gender_acc(gender_logits, genders)
            gender_acc = self.val_gender_acc.compute()
        
        age_probs = F.softmax(age_logits, dim=1)
        pred_ages = (age_probs * self.age_bins.to(age_probs.device)).sum(dim=1)
        true_ages = (age_targets * self.age_bins.to(age_targets.device)).sum(dim=1)
        age_rmse = torch.sqrt(F.mse_loss(pred_ages, true_ages))
        age_mae = F.l1_loss(pred_ages, true_ages)
        
        self.log(f"{stage}_loss", total_loss, prog_bar=True)
        self.log(f"{stage}_gender_loss", gender_loss, prog_bar=True)
        self.log(f"{stage}_age_loss", age_loss, prog_bar=True)
        self.log(f"{stage}_gender_acc", gender_acc, prog_bar=True)
        self.log(f"{stage}_age_rmse", age_rmse, prog_bar=True)
        self.log(f"{stage}_age_mae", age_mae, prog_bar=True)
        
        return total_loss

    def training_step(self, batch, batch_idx):
        return self._shared_step(batch, "train")

    def validation_step(self, batch, batch_idx):
        self._shared_step(batch, "val")

## Data Pipeline

In [6]:

class EnhancedFaceDataset(Dataset):
    def __init__(self, df, root_dir, transforms=None, train=True):
        self.df = df.reset_index(drop=True)
        self.root = Path(root_dir)
        self.transforms = transforms
        self.train = train
        
        if train and 'age' in df.columns:
            self.soft_labels = self._create_soft_labels(df['age'].values)
        else:
            self.soft_labels = None

    def _create_soft_labels(self, ages, sigma=1.5):
        age_bins = np.arange(0, Config.NUM_AGE_BINS)
        soft_labels = []
        for age in ages:
            probs = np.exp(-0.5 * ((age_bins - age) / sigma) ** 2)
            soft_labels.append((probs / probs.sum()).astype(np.float32))
        return np.vstack(soft_labels)

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        img_path = self.root / row['full_path']
        
        try:
            img = Image.open(img_path).convert("RGB")
        except:
            img = Image.new('RGB', (Config.IMG_SIZE, Config.IMG_SIZE), color='black')
        
        if self.transforms:
            img = self.transforms(img)
            
        if self.train:
            gender = torch.tensor(row['gender'], dtype=torch.long)
            age_target = torch.tensor(self.soft_labels[idx], dtype=torch.float32)
            return img, gender, age_target
        else:
            return img, row['id']

class TestFaceDataset(Dataset):
    def __init__(self, test_csv, root_dir, transforms=None):
        self.df = pd.read_csv(test_csv)
        self.root = Path(root_dir)
        self.transforms = transforms

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        img_path = self.root / row['full_path']
        try:
            img = Image.open(img_path).convert("RGB")
        except:
            img = Image.new('RGB', (Config.IMG_SIZE, Config.IMG_SIZE), color='black')
        if self.transforms:
            img = self.transforms(img)
        return img, row['id']

def get_enhanced_transforms():
    train_transforms = transforms.Compose([
        transforms.Resize((Config.IMG_SIZE, Config.IMG_SIZE)),
        transforms.RandomHorizontalFlip(p=0.5),
        transforms.ColorJitter(0.2, 0.2, 0.2, 0.1),
        transforms.RandomRotation(5),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ])
    
    val_transforms = transforms.Compose([
        transforms.Resize((Config.IMG_SIZE, Config.IMG_SIZE)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])
    
    return train_transforms, val_transforms

def get_enhanced_dataloaders(batch_size=Config.BATCH_SIZE):
    df = pd.read_csv(Config.TRAIN_CSV)
    
    df['age_bucket'] = pd.cut(df['age'], bins=10, labels=False)
    df['stratify_label'] = df['age_bucket'].astype(str) + "_" + df['gender'].astype(str)
    
    train_df, val_df = train_test_split(
        df, test_size=0.15, random_state=Config.SEED,
        stratify=df['stratify_label']
    )
    
    train_transforms, val_transforms = get_enhanced_transforms()
    
    train_ds = EnhancedFaceDataset(train_df, Config.ROOT_DIR, train_transforms, train=True)
    val_ds = EnhancedFaceDataset(val_df, Config.ROOT_DIR, val_transforms, train=True)
    
    train_loader = DataLoader(train_ds, batch_size=batch_size, shuffle=True, num_workers=2)
    val_loader = DataLoader(val_ds, batch_size=batch_size, shuffle=False, num_workers=2)
    
    trackio.log({
        "training_samples": len(train_ds),
        "validation_samples": len(val_ds),
        "total_samples": len(df)
    })
    
    print(f"📊 Training samples: {len(train_ds)}")
    print(f"📊 Validation samples: {len(val_ds)}")
    
    return train_loader, val_loader

print("✅ Data pipeline ready!")

✅ Data pipeline ready!


## Training Setup & Kaggle Hub Utilities

In [7]:
class TrackIOCallback(Callback):
    def on_train_start(self, trainer, pl_module):
        trackio.log({"status": "training_started"})
        
    def on_train_end(self, trainer, pl_module):
        trackio.log({"status": "training_completed"})
        
    def on_epoch_end(self, trainer, pl_module):
        metrics = {
            "epoch": trainer.current_epoch,
            "train_loss": trainer.callback_metrics.get('train_loss', torch.tensor(0.0)).item(),
            "val_loss": trainer.callback_metrics.get('val_loss', torch.tensor(0.0)).item(),
            "val_gender_acc": trainer.callback_metrics.get('val_gender_acc', torch.tensor(0.0)).item(),
            "val_age_mae": trainer.callback_metrics.get('val_age_mae', torch.tensor(0.0)).item(),
        }
        trackio.log(metrics)

def save_model_to_kaggle_hub(model, model_name="face-age-gender-Efficientnet-B4-model"):
    try:
        model_dir = Path("/kaggle/working/model_artifacts")
        model_dir.mkdir(exist_ok=True)
        
        model_path = model_dir / "model_weights.pth"
        torch.save(model.state_dict(), model_path)
        
        model_info = {
            "model_name": model_name,
            "backbone": "tf_efficientnet_b4_ns",
            "img_size": Config.IMG_SIZE,
            "num_age_bins": Config.NUM_AGE_BINS,
            "timestamp": datetime.now().isoformat()
        }
        
        with open(model_dir / "model_info.json", "w") as f:
            json.dump(model_info, f, indent=2)
        
        print("✅ Model saved locally for Kaggle Hub")
        return model_dir
        
    except Exception as e:
        print(f"❌ Failed to save model: {e}")
        return None

def load_model_from_kaggle_hub(model_identifier, model_class):
    try:
        print(f"📥 Downloading model from Kaggle Hub: {model_identifier}")
        
        model_path = kagglehub.model_download(model_identifier)
        print(f"✅ Model downloaded to: {model_path}")
        
        weights_path = Path(model_path) / "model_weights.pth"
        model_info_path = Path(model_path) / "model_info.json"
        
        if weights_path.exists():
            model = model_class()
            model.load_state_dict(torch.load(weights_path, map_location=Config.DEVICE))
            model.to(Config.DEVICE)
            model.eval()
            
            if model_info_path.exists():
                with open(model_info_path, 'r') as f:
                    model_info = json.load(f)
                print(f"📋 Model info: {model_info}")
            
            print("✅ Model loaded successfully from Kaggle Hub")
            return model
        else:
            print("❌ Model weights not found")
            return None
            
    except Exception as e:
        print(f"❌ Failed to load model from Kaggle Hub: {e}")
        return None

# 🔧 ADD THIS LINE BREAK - THIS WAS MISSING!
class EnhancedTrackIOCallback(Callback):
    def on_train_start(self, trainer, pl_module):
        trackio.log({"status": "training_started", "model_type": "efficientnet_b4"})
        
    def on_train_end(self, trainer, pl_module):
        trackio.log({"status": "training_completed"})
        
    def on_epoch_end(self, trainer, pl_module):
        # 🔧 ENHANCED: Log comprehensive metrics for better Hugging Face graphs
        metrics = {
            "epoch": trainer.current_epoch,
            
            # Loss metrics
            "train_loss": trainer.callback_metrics.get('train_loss', torch.tensor(0.0)).item(),
            "val_loss": trainer.callback_metrics.get('val_loss', torch.tensor(0.0)).item(),
            "train_gender_loss": trainer.callback_metrics.get('train_gender_loss', torch.tensor(0.0)).item(),
            "val_gender_loss": trainer.callback_metrics.get('val_gender_loss', torch.tensor(0.0)).item(),
            "train_age_loss": trainer.callback_metrics.get('train_age_loss', torch.tensor(0.0)).item(),
            "val_age_loss": trainer.callback_metrics.get('val_age_loss', torch.tensor(0.0)).item(),
            
            # Gender metrics
            "train_gender_acc": trainer.callback_metrics.get('train_gender_acc', torch.tensor(0.0)).item(),
            "val_gender_acc": trainer.callback_metrics.get('val_gender_acc', torch.tensor(0.0)).item(),
            
            # Age metrics
            "train_age_mae": trainer.callback_metrics.get('train_age_mae', torch.tensor(0.0)).item(),
            "val_age_mae": trainer.callback_metrics.get('val_age_mae', torch.tensor(0.0)).item(),
            "train_age_rmse": trainer.callback_metrics.get('train_age_rmse', torch.tensor(0.0)).item(),
            "val_age_rmse": trainer.callback_metrics.get('val_age_rmse', torch.tensor(0.0)).item(),
            
            # Learning rate
            "learning_rate": trainer.optimizers[0].param_groups[0]['lr']
        }
        trackio.log(metrics)

##  Training Execution

In [8]:
def train_fixed_model():
    print("🚀 Starting Fixed EfficientNet-B4 Training...")
    
    trackio.log({
        "training_status": "starting",
        "model_architecture": "efficientnet_b4_with_attention",
        "total_epochs": Config.EPOCHS,
        "batch_size": Config.BATCH_SIZE,
        "image_size": Config.IMG_SIZE
    })
    
    train_loader, val_loader = get_enhanced_dataloaders()
    
    model = FixedFaceAgeGenderModel(lr=Config.LEARNING_RATE)
    
    print("🧪 Testing forward pass...")
    try:
        dummy_input = torch.randn(2, 3, Config.IMG_SIZE, Config.IMG_SIZE)
        gender_out, age_out = model(dummy_input)
        print(f"✅ Forward pass successful!")
        
        trackio.log({
            "model_backbone": "tf_efficientnet_b4_ns",
            "feature_dim": model.backbone.num_features,
            "gender_output_dim": gender_out.shape[1],
            "age_output_dim": age_out.shape[1],
            "total_parameters": sum(p.numel() for p in model.parameters()),
            "trainable_parameters": sum(p.numel() for p in model.parameters() if p.requires_grad)
        })
        
    except Exception as e:
        print(f"❌ Forward pass failed: {e}")
        trackio.log({"error": f"Forward pass failed: {e}"})
        return None, None, None
    
    checkpoint_callback = ModelCheckpoint(
        monitor='val_gender_acc',
        dirpath='/kaggle/working/fixed_face_model',
        filename='fixed-best-{epoch:02d}-{val_gender_acc:.3f}',
        save_top_k=1,
        mode='max'
    )
    
    early_stop_callback = EarlyStopping(
        monitor='val_loss',
        min_delta=0.001,
        patience=8,
        mode='min'
    )
    
    # 🔧 USING ENHANCED CALLBACK
    track_callback = EnhancedTrackIOCallback()
    
    trainer = pl.Trainer(
        max_epochs=Config.EPOCHS,
        accelerator="auto",
        devices="auto",
        callbacks=[checkpoint_callback, early_stop_callback, track_callback],
        log_every_n_steps=10,
    )
    
    print("📈 Starting training...")
    trackio.log({"training_status": "in_progress"})
    
    trainer.fit(model, train_loader, val_loader)
    
    model_dir = save_model_to_kaggle_hub(model)
    
    final_metrics = {
        "final_train_loss": trainer.callback_metrics.get('train_loss', torch.tensor(0.0)).item(),
        "final_val_loss": trainer.callback_metrics.get('val_loss', torch.tensor(0.0)).item(),
        "final_val_gender_acc": trainer.callback_metrics.get('val_gender_acc', torch.tensor(0.0)).item(),
        "final_val_age_mae": trainer.callback_metrics.get('val_age_mae', torch.tensor(0.0)).item(),
        "final_val_age_rmse": trainer.callback_metrics.get('val_age_rmse', torch.tensor(0.0)).item(),
        "training_status": "completed",
        "best_checkpoint": checkpoint_callback.best_model_path if checkpoint_callback.best_model_path else "none"
    }
    trackio.log(final_metrics)
    
    print("✅ Fixed training completed!")
    return trainer, model, model_dir

# COMMENT THIS IN FINAL SUBMISSION:
trainer, model, model_dir = train_fixed_model()

🚀 Starting Fixed EfficientNet-B4 Training...
📊 Training samples: 29501
📊 Validation samples: 5207


model.safetensors:   0%|          | 0.00/77.9M [00:00<?, ?B/s]

📊 Backbone feature dimension: 1792
🧪 Testing forward pass...


Trainer will use only 1 of 2 GPUs because it is running inside an interactive / notebook environment. You may try to set `Trainer(devices=2)` but please note that multi-GPU inside interactive / notebook environments is considered experimental and unstable. Your mileage may vary.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


✅ Forward pass successful!
📈 Starting training...


...

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

`Trainer.fit` stopped: `max_epochs=5` reached.


✅ Model saved locally for Kaggle Hub
✅ Fixed training completed!


## Simple CNN Training

In [9]:
def train_simple_cnn_model():
    print("🚀 Starting Simple CNN Training...")
    
    # 🔧 ENHANCED: Log Simple CNN specific info
    trackio.log({
        "training_status": "starting",
        "model_architecture": "simple_cnn_scratch",
        "model_type": "from_scratch",
        "total_epochs": Config.EPOCHS,
        "batch_size": Config.BATCH_SIZE
    })
    
    train_loader, val_loader = get_enhanced_dataloaders()
    
    model = SimpleFaceAgeGenderModel(lr=1e-3)
    
    print("🧪 Testing forward pass...")
    try:
        dummy_input = torch.randn(2, 3, Config.IMG_SIZE, Config.IMG_SIZE)
        gender_out, age_out = model(dummy_input)
        print(f"✅ Forward pass successful!")
        print(f"   Gender output shape: {gender_out.shape}")
        print(f"   Age output shape: {age_out.shape}")
        
        # 🔧 ENHANCED: Log Simple CNN architecture details
        trackio.log({
            "model_type": "simple_cnn",
            "total_parameters": sum(p.numel() for p in model.parameters()),
            "trainable_parameters": sum(p.numel() for p in model.parameters() if p.requires_grad),
            "gender_output_dim": gender_out.shape[1],
            "age_output_dim": age_out.shape[1],
            "is_scratch_model": True
        })
        
    except Exception as e:
        print(f"❌ Forward pass failed: {e}")
        trackio.log({"error": f"Forward pass failed: {e}"})
        return None, None, None
    
    checkpoint_callback = ModelCheckpoint(
        monitor='val_gender_acc',
        dirpath='/kaggle/working/simple_cnn_model',
        filename='simple-cnn-best-{epoch:02d}-{val_gender_acc:.3f}',
        save_top_k=1,
        mode='max'
    )
    
    early_stop_callback = EarlyStopping(
        monitor='val_loss',
        min_delta=0.001,
        patience=5,
        mode='min'
    )
    
    # 🔧 USING ENHANCED CALLBACK FOR SIMPLE CNN TOO
    track_callback = EnhancedTrackIOCallback()
    
    trainer = pl.Trainer(
        max_epochs=Config.EPOCHS,
        accelerator="auto",
        devices="auto",
        callbacks=[checkpoint_callback, early_stop_callback, track_callback],
        log_every_n_steps=10,
    )
    
    print("📈 Starting training...")
    trackio.log({"training_status": "in_progress"})
    
    trainer.fit(model, train_loader, val_loader)
    
    model_dir = save_model_to_kaggle_hub(model, "simple-cnn-face-model")
    
    # 🔧 ENHANCED: Log Simple CNN final metrics
    final_metrics = {
        "final_train_loss": trainer.callback_metrics.get('train_loss', torch.tensor(0.0)).item(),
        "final_val_loss": trainer.callback_metrics.get('val_loss', torch.tensor(0.0)).item(),
        "final_val_gender_acc": trainer.callback_metrics.get('val_gender_acc', torch.tensor(0.0)).item(),
        "final_val_age_mae": trainer.callback_metrics.get('val_age_mae', torch.tensor(0.0)).item(),
        "training_status": "completed",
        "model_type": "simple_cnn",
        "best_checkpoint": checkpoint_callback.best_model_path if checkpoint_callback.best_model_path else "none"
    }
    trackio.log(final_metrics)
    
    print("✅ Simple CNN training completed!")
    return trainer, model, model_dir

# COMMENT THIS IN FINAL SUBMISSION:
simple_trainer, simple_model, simple_model_dir = train_simple_cnn_model()

🚀 Starting Simple CNN Training...
📊 Training samples: 29501
📊 Validation samples: 5207
🧪 Testing forward pass...


Trainer will use only 1 of 2 GPUs because it is running inside an interactive / notebook environment. You may try to set `Trainer(devices=2)` but please note that multi-GPU inside interactive / notebook environments is considered experimental and unstable. Your mileage may vary.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]

  | Name             | Type               | Params | Mode 
----------------------------------------------------------------
0 | cnn              | SimpleCNN          | 5.0 M  | train
1 | gender_loss_fn   | CrossEntropyLoss   | 0      | train
2 | age_loss_fn      | KLDivLoss          | 0      | train
3 | train_gender_acc | MulticlassAccuracy | 0      | train
4 | val_gender_acc   | MulticlassAccuracy | 0      | train
----------------------------------------------------------------
5.0 M     Trainable params
0         Non-trainable params
5.0 M     Total p

✅ Forward pass successful!
   Gender output shape: torch.Size([2, 2])
   Age output shape: torch.Size([2, 101])
📈 Starting training...


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

...

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

`Trainer.fit` stopped: `max_epochs=5` reached.


✅ Model saved locally for Kaggle Hub
✅ Simple CNN training completed!


## Test Dataset & Inference Setup 

In [10]:
class TestFaceDataset(Dataset):
    def __init__(self, test_csv, root_dir, transforms=None):
        self.df = pd.read_csv(test_csv)
        self.root = Path(root_dir)
        self.transforms = transforms

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        img_path = self.root / row['full_path']
        try:
            img = Image.open(img_path).convert("RGB")
        except:
            img = Image.new('RGB', (Config.IMG_SIZE, Config.IMG_SIZE), color='black')
        if self.transforms:
            img = self.transforms(img)
        return img, row['id']

In [11]:
def enhanced_predict(use_kaggle_hub=False, model_identifier=None):
    """Make predictions with option to load from Kaggle Hub"""
    
    trackio.log({"inference_status": "starting", "model_type": "efficientnet_b4"})
    
    if use_kaggle_hub and model_identifier:
        print("🔄 Loading model from Kaggle Hub...")
        model = load_model_from_kaggle_hub(model_identifier, FixedFaceAgeGenderModel)
        if model is None:
            print("❌ Failed to load from Kaggle Hub, using local model")
            if 'trainer' not in globals() or trainer is None:
                print("❌ No local model available")
                return None
            checkpoint_path = trainer.checkpoint_callback.best_model_path
            model = FixedFaceAgeGenderModel.load_from_checkpoint(checkpoint_path)
    else:
        if 'trainer' not in globals() or trainer is None:
            print("❌ Training failed, cannot make predictions")
            return None
            
        checkpoint_path = trainer.checkpoint_callback.best_model_path
        print(f"📁 Loading best checkpoint: {checkpoint_path}")
        model = FixedFaceAgeGenderModel.load_from_checkpoint(checkpoint_path)
    
    model.to(Config.DEVICE)
    model.eval()
    
    # Prepare test data
    _, test_transforms = get_enhanced_transforms()
    test_ds = TestFaceDataset(Config.TEST_CSV, Config.ROOT_DIR, test_transforms)
    test_loader = DataLoader(test_ds, batch_size=Config.BATCH_SIZE, shuffle=False, num_workers=2)
    
    all_ids = []
    all_genders = []
    all_ages = []
    
    print("🔮 Making predictions...")
    trackio.log({"inference_status": "in_progress"})
    
    with torch.no_grad():
        for batch in tqdm(test_loader, desc="Predicting"):
            imgs, ids = batch
            imgs = imgs.to(Config.DEVICE)
            
            gender_logits, age_logits = model(imgs)
            
            gender_preds = torch.argmax(gender_logits, dim=1).cpu().numpy()
            age_probs = F.softmax(age_logits, dim=1).cpu().numpy()
            age_preds = (age_probs * np.arange(0, Config.NUM_AGE_BINS)).sum(axis=1)
            
            if isinstance(ids, torch.Tensor):
                ids = ids.cpu().numpy().tolist()
            
            all_ids.extend(ids)
            all_genders.extend(gender_preds)
            all_ages.extend(age_preds)
    
    # Create submission
    submission = pd.DataFrame({
        'id': all_ids,
        'age': np.clip(all_ages, 0, 100).round().astype(int),
        'gender': all_genders
    })
    
    # 🔧 ENHANCED: Log comprehensive inference statistics
    inference_stats = {
        "test_samples_processed": len(submission),
        "predicted_age_mean": float(submission['age'].mean()),
        "predicted_age_std": float(submission['age'].std()),
        "predicted_age_min": int(submission['age'].min()),
        "predicted_age_max": int(submission['age'].max()),
        "male_predictions": int((submission['gender'] == 0).sum()),
        "female_predictions": int((submission['gender'] == 1).sum()),
        "gender_ratio": float((submission['gender'] == 1).sum() / len(submission)),
        "inference_status": "completed",
        "model_used": "efficientnet_b4"
    }
    trackio.log(inference_stats)
    
    print(f"✅ Inference completed: {len(submission)} samples processed")
    return submission

In [12]:
def simple_cnn_predict(use_kaggle_hub=False, model_identifier=None):
    """Make predictions with Simple CNN model"""
    
    trackio.log({"inference_status": "starting", "model_type": "simple_cnn"})
    
    if use_kaggle_hub and model_identifier:
        print("🔄 Loading Simple CNN model from Kaggle Hub...")
        model = load_model_from_kaggle_hub(model_identifier, SimpleFaceAgeGenderModel)
        if model is None:
            print("❌ Failed to load from Kaggle Hub")
            return None
    else:
        # You need to train the simple CNN first or load from checkpoint
        print("❌ No Simple CNN model available. Please train first or provide Kaggle Hub identifier.")
        return None
    
    model.to(Config.DEVICE)
    model.eval()
    
    # Prepare test data
    _, test_transforms = get_enhanced_transforms()
    test_ds = TestFaceDataset(Config.TEST_CSV, Config.ROOT_DIR, test_transforms)
    test_loader = DataLoader(test_ds, batch_size=Config.BATCH_SIZE, shuffle=False, num_workers=2)
    
    all_ids = []
    all_genders = []
    all_ages = []
    
    print("🔮 Making predictions with Simple CNN...")
    trackio.log({"inference_status": "in_progress", "model_type": "simple_cnn"})
    
    with torch.no_grad():
        for batch in tqdm(test_loader, desc="Predicting with Simple CNN"):
            imgs, ids = batch
            imgs = imgs.to(Config.DEVICE)
            
            gender_logits, age_logits = model(imgs)
            
            gender_preds = torch.argmax(gender_logits, dim=1).cpu().numpy()
            age_probs = F.softmax(age_logits, dim=1).cpu().numpy()
            age_preds = (age_probs * np.arange(0, Config.NUM_AGE_BINS)).sum(axis=1)
            
            if isinstance(ids, torch.Tensor):
                ids = ids.cpu().numpy().tolist()
            
            all_ids.extend(ids)
            all_genders.extend(gender_preds)
            all_ages.extend(age_preds)
    
    # Create submission
    submission = pd.DataFrame({
        'id': all_ids,
        'age': np.clip(all_ages, 0, 100).round().astype(int),
        'gender': all_genders
    })
    
    # 🔧 ENHANCED: Log Simple CNN inference statistics
    inference_stats = {
        "test_samples_processed": len(submission),
        "predicted_age_mean": float(submission['age'].mean()),
        "predicted_age_std": float(submission['age'].std()),
        "predicted_age_min": int(submission['age'].min()),
        "predicted_age_max": int(submission['age'].max()),
        "male_predictions": int((submission['gender'] == 0).sum()),
        "female_predictions": int((submission['gender'] == 1).sum()),
        "gender_ratio": float((submission['gender'] == 1).sum() / len(submission)),
        "inference_status": "completed",
        "model_used": "simple_cnn_scratch"
    }
    trackio.log(inference_stats)
    
    print(f"✅ Simple CNN inference completed: {len(submission)} samples processed")
    return submission

In [13]:
# Final Inference & Results
print("🚀 Starting Final Inference Pipeline...")

# Option 1: Use EfficientNet model (your main model)
print("### Using EfficientNet-B4 Model ###")
enhanced_submission = enhanced_predict(use_kaggle_hub=False)

if enhanced_submission is not None:
    # Save main submission
    enhanced_submission.to_csv('submission.csv', index=False)
    
    print("\n✅ Enhanced inference completed!")
    print(f"📊 Processed {len(enhanced_submission)} test samples")
    print("\n📋 Submission preview:")
    print(enhanced_submission.head(10))
    
    # Show statistics
    print(f"\n📈 Submission Statistics:")
    print(f"Age range: {enhanced_submission['age'].min()} - {enhanced_submission['age'].max()}")
    print(f"Mean age: {enhanced_submission['age'].mean():.1f}")
    print(f"Gender distribution:")
    print(f"  Male (0): {(enhanced_submission['gender'] == 0).sum()}")
    print(f"  Female (1): {(enhanced_submission['gender'] == 1).sum()}")
    
    # 🔧 ENHANCED: Log final submission details
    trackio.log({
        "final_submission_ready": True,
        "submission_file": "submission.csv",
        "total_predictions": len(enhanced_submission),
        "final_age_range": f"{enhanced_submission['age'].min()}-{enhanced_submission['age'].max()}",
        "final_gender_balance": f"{(enhanced_submission['gender'] == 1).mean():.2f}"
    })
    
else:
    print("❌ EfficientNet inference failed, trying Simple CNN...")
    
    # Option 2: Fallback to Simple CNN
    print("### Trying Simple CNN Model ###")
    simple_submission = simple_cnn_predict(use_kaggle_hub=False)
    
    if simple_submission is not None:
        simple_submission.to_csv('submission.csv', index=False)
        print("✅ Simple CNN submission saved as fallback!")
        
        trackio.log({
            "final_submission_ready": True,
            "submission_file": "submission.csv", 
            "model_used": "simple_cnn_fallback",
            "total_predictions": len(simple_submission)
        })
    else:
        print("❌ All models failed, creating random fallback submission...")
        
        # Final fallback
        test_df = pd.read_csv(Config.TEST_CSV)
        enhanced_submission = pd.DataFrame({
            'id': test_df['id'].values,
            'age': np.random.randint(20, 60, len(test_df)),
            'gender': np.random.randint(0, 2, len(test_df))
        })
        enhanced_submission.to_csv('submission.csv', index=False)
        print("✅ Random fallback submission created")
        
        trackio.log({
            "final_submission_ready": True,
            "submission_file": "submission.csv",
            "model_used": "random_fallback",
            "total_predictions": len(enhanced_submission)
        })

# Finish Track.io run
try:
    trackio.finish()
    print("✅ Track.io logging completed successfully!")
except Exception as e:
    print(f"⚠️ Track.io finish had an issue (non-critical): {e}")

print("🎯 COMPETITION PIPELINE COMPLETED! 🎯")
print("📁 submission.csv is ready for Kaggle submission")
print("📊 Check Hugging Face Space for training graphs and metrics")

🚀 Starting Final Inference Pipeline...
### Using EfficientNet-B4 Model ###
📁 Loading best checkpoint: /kaggle/working/fixed_face_model/fixed-best-epoch=04-val_gender_acc=0.968.ckpt
📊 Backbone feature dimension: 1792
🔮 Making predictions...


Predicting: 100%|██████████| 543/543 [01:36<00:00,  5.64it/s]


✅ Inference completed: 8677 samples processed

✅ Enhanced inference completed!
📊 Processed 8677 test samples

📋 Submission preview:
   id  age  gender
0   0   33       1
1   1   26       0
2   2   23       1
3   3   35       1
4   4   76       1
5   5   72       1
6   6   30       1
7   7   25       1
8   8   27       0
9   9   46       0

📈 Submission Statistics:
Age range: 17 - 81
Mean age: 36.5
Gender distribution:
  Male (0): 2178
  Female (1): 6499
* Run finished. Uploading logs to Trackio (please wait...)
✅ Track.io logging completed successfully!
🎯 COMPETITION PIPELINE COMPLETED! 🎯
📁 submission.csv is ready for Kaggle submission
📊 Check Hugging Face Space for training graphs and metrics


In [14]:
print("=== DEBUGGING ===")
print("1. Testing imports...")
import torch
print(f"   PyTorch: {torch.__version__}")

print("2. Testing configuration...")
print(f"   Device: {Config.DEVICE}")
print(f"   IMG_SIZE: {Config.IMG_SIZE}")

print("3. Testing data access...")
try:
    import pandas as pd
    df = pd.read_csv(Config.TRAIN_CSV)
    print(f"   ✅ Data: {len(df)} samples")
except Exception as e:
    print(f"   ❌ Data error: {e}")

print("4. Testing inference...")
try:
    # Try to run a small part of inference
    from EnhancedFaceDataset import EnhancedFaceDataset
    print("   ✅ Data classes imported")
except Exception as e:
    print(f"   ❌ Import error: {e}")

print("=== DEBUG COMPLETE ===")

=== DEBUGGING ===
1. Testing imports...
   PyTorch: 2.6.0+cu124
2. Testing configuration...
   Device: cuda
   IMG_SIZE: 384
3. Testing data access...
   ✅ Data: 34708 samples
4. Testing inference...
   ❌ Import error: No module named 'EnhancedFaceDataset'
=== DEBUG COMPLETE ===


In [15]:
print("=== FINAL TEST ===")
try:
    # Test dataset creation
    _, test_transforms = get_enhanced_transforms()
    test_ds = TestFaceDataset(Config.TEST_CSV, Config.ROOT_DIR, test_transforms)
    print(f"✅ Test dataset: {len(test_ds)} samples")
    
    # Test model loading
    model = FixedFaceAgeGenderModel()
    print("✅ Model loaded")
    
    print("🎯 Ready for inference!")
    
except Exception as e:
    print(f"❌ Error: {e}")

=== FINAL TEST ===
✅ Test dataset: 8677 samples
📊 Backbone feature dimension: 1792
✅ Model loaded
🎯 Ready for inference!


In [16]:
# run = trackio.init(
#     project="25-t3-nppe1",
#     name=f"EB4-{datetime.now().strftime('%Y-%m-%d-%H-%M-%S')}",
#     group="efficientnet-b4-training",
#     space_id="Abhishek-A0/dlgenai-nppe"
# )

In [17]:
# SEED = 42
# torch.manual_seed(SEED)
# np.random.seed(SEED)

# IMG_SIZE = 384
# BATCH_SIZE = 16
# NUM_AGE_BINS = 101
# EPOCHS = 1
# DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# print(f" Device: {DEVICE}")

In [18]:
# trackio.log({
#     "seed": SEED,
#     "img_size": IMG_SIZE,
#     "batch_size": BATCH_SIZE,
#     "num_age_bins": NUM_AGE_BINS,
#     "epochs": EPOCHS,
#     "device": str(DEVICE),
#     "model_backbone": "tf_efficientnet_b4_ns",
#     "learning_rate": 2e-4,
#     "optimizer": "AdamW",
#     "scheduler": "CosineAnnealingLR"
# })


In [19]:
# TRAIN_CSV = "/kaggle/input/sep-25-dl-gen-ai-nppe-1/face_dataset/train.csv"
# TEST_CSV = "/kaggle/input/sep-25-dl-gen-ai-nppe-1/face_dataset/test.csv"
# ROOT_DIR = "/kaggle/input/sep-25-dl-gen-ai-nppe-1/face_dataset"

In [20]:

# class FocalLoss(nn.Module):
#     def __init__(self, alpha=0.5, gamma=2.0):
#         super().__init__()
#         self.alpha = alpha
#         self.gamma = gamma
        
#     def forward(self, inputs, targets):
#         BCE_loss = F.cross_entropy(inputs, targets, reduction='none')
#         pt = torch.exp(-BCE_loss)
#         focal_loss = self.alpha * (1-pt)**self.gamma * BCE_loss
#         return focal_loss.mean()

# class FixedFaceAgeGenderModel(pl.LightningModule):
#     def __init__(self, backbone_name="tf_efficientnet_b4_ns", num_age_bins=101, lr=2e-4):
#         super().__init__()
#         self.save_hyperparameters()
        
#         # Backbone with proper global pooling
#         self.backbone = timm.create_model(
#             backbone_name, 
#             pretrained=True,
#             num_classes=0,
#             global_pool='avg'
#         )
#         feat_dim = self.backbone.num_features
        
#         print(f"📊 Backbone feature dimension: {feat_dim}")
        
#         self.attention = nn.Sequential(
#             nn.Linear(feat_dim, 256),
#             nn.ReLU(),
#             nn.Dropout(0.2),
#             nn.Linear(256, feat_dim),
#             nn.Sigmoid()
#         )
        
#         self.gender_head = nn.Sequential(
#             nn.Linear(feat_dim, 256),
#             nn.BatchNorm1d(256),
#             nn.ReLU(),
#             nn.Dropout(0.3),
#             nn.Linear(256, 128),
#             nn.ReLU(),
#             nn.Dropout(0.2),
#             nn.Linear(128, 2)
#         )
        
#         self.age_head = nn.Sequential(
#             nn.Linear(feat_dim, 256),
#             nn.BatchNorm1d(256),
#             nn.ReLU(),
#             nn.Dropout(0.3),
#             nn.Linear(256, 128),
#             nn.ReLU(),
#             nn.Dropout(0.2),
#             nn.Linear(128, num_age_bins)
#         )
        
#         self.gender_loss_fn = nn.CrossEntropyLoss()
#         self.age_loss_fn = nn.KLDivLoss(reduction='batchmean')
        
#         self.train_gender_acc = torchmetrics.classification.Accuracy(task="multiclass", num_classes=2)
#         self.val_gender_acc = torchmetrics.classification.Accuracy(task="multiclass", num_classes=2)
        
#         self.age_bins = torch.arange(0, num_age_bins).float()

#     def forward(self, x):
#         features = self.backbone(x)
#         attention_weights = self.attention(features)
#         attended_features = features * attention_weights
        
#         gender_logits = self.gender_head(attended_features)
#         age_logits = self.age_head(attended_features)
        
#         return gender_logits, age_logits

#     def configure_optimizers(self):
#         optimizer = torch.optim.AdamW(
#             self.parameters(), 
#             lr=self.hparams.lr,
#             weight_decay=0.01,
#         )
        
#         scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(
#             optimizer, 
#             T_max=EPOCHS
#         )
        
#         return [optimizer], [scheduler]

#     def _shared_step(self, batch, stage):
#         imgs, genders, age_targets = batch
#         gender_logits, age_logits = self(imgs)
        
#         gender_loss = self.gender_loss_fn(gender_logits, genders)
#         age_loss = self.age_loss_fn(F.log_softmax(age_logits, dim=1), age_targets)
#         total_loss = gender_loss + age_loss
        
#         # Gender metrics
#         if stage == 'train':
#             self.train_gender_acc(gender_logits, genders)
#             gender_acc = self.train_gender_acc.compute()
#         else:
#             self.val_gender_acc(gender_logits, genders)
#             gender_acc = self.val_gender_acc.compute()
        
#         # Age metrics
#         age_probs = F.softmax(age_logits, dim=1)
#         pred_ages = (age_probs * self.age_bins.to(age_probs.device)).sum(dim=1)
#         true_ages = (age_targets * self.age_bins.to(age_targets.device)).sum(dim=1)
#         age_rmse = torch.sqrt(F.mse_loss(pred_ages, true_ages))
#         age_mae = F.l1_loss(pred_ages, true_ages)
        
#         # Log metrics to PyTorch Lightning
#         self.log(f"{stage}_loss", total_loss, prog_bar=True)
#         self.log(f"{stage}_gender_loss", gender_loss, prog_bar=True)
#         self.log(f"{stage}_age_loss", age_loss, prog_bar=True)
#         self.log(f"{stage}_gender_acc", gender_acc, prog_bar=True)
#         self.log(f"{stage}_age_rmse", age_rmse, prog_bar=True)
#         self.log(f"{stage}_age_mae", age_mae, prog_bar=True)
        
#         return total_loss

#     def training_step(self, batch, batch_idx):
#         return self._shared_step(batch, "train")

#     def validation_step(self, batch, batch_idx):
#         self._shared_step(batch, "val")

In [21]:
# # Cell 3b: Simple CNN Model Definition

# class SimpleCNN(nn.Module):
#     def __init__(self, num_age_bins=101):
#         super().__init__()
        
#         # Feature extraction backbone
#         self.features = nn.Sequential(
#             # First conv block
#             nn.Conv2d(3, 64, kernel_size=3, padding=1),
#             nn.BatchNorm2d(64),
#             nn.ReLU(inplace=True),
#             nn.Conv2d(64, 64, kernel_size=3, padding=1),
#             nn.BatchNorm2d(64),
#             nn.ReLU(inplace=True),
#             nn.MaxPool2d(kernel_size=2, stride=2),
#             nn.Dropout2d(0.25),
            
#             # Second conv block
#             nn.Conv2d(64, 128, kernel_size=3, padding=1),
#             nn.BatchNorm2d(128),
#             nn.ReLU(inplace=True),
#             nn.Conv2d(128, 128, kernel_size=3, padding=1),
#             nn.BatchNorm2d(128),
#             nn.ReLU(inplace=True),
#             nn.MaxPool2d(kernel_size=2, stride=2),
#             nn.Dropout2d(0.25),
            
#             # Third conv block
#             nn.Conv2d(128, 256, kernel_size=3, padding=1),
#             nn.BatchNorm2d(256),
#             nn.ReLU(inplace=True),
#             nn.Conv2d(256, 256, kernel_size=3, padding=1),
#             nn.BatchNorm2d(256),
#             nn.ReLU(inplace=True),
#             nn.MaxPool2d(kernel_size=2, stride=2),
#             nn.Dropout2d(0.25),
            
#             # Fourth conv block
#             nn.Conv2d(256, 512, kernel_size=3, padding=1),
#             nn.BatchNorm2d(512),
#             nn.ReLU(inplace=True),
#             nn.Conv2d(512, 512, kernel_size=3, padding=1),
#             nn.BatchNorm2d(512),
#             nn.ReLU(inplace=True),
#             nn.AdaptiveAvgPool2d((1, 1)),
#             nn.Dropout2d(0.25),
#         )
        
#         # Classifiers
#         self.gender_classifier = nn.Sequential(
#             nn.Dropout(0.5),
#             nn.Linear(512, 256),
#             nn.ReLU(inplace=True),
#             nn.Dropout(0.5),
#             nn.Linear(256, 2)
#         )
        
#         self.age_classifier = nn.Sequential(
#             nn.Dropout(0.5),
#             nn.Linear(512, 256),
#             nn.ReLU(inplace=True),
#             nn.Dropout(0.5),
#             nn.Linear(256, num_age_bins)
#         )
        
#     def forward(self, x):
#         features = self.features(x)
#         features = features.view(features.size(0), -1)
        
#         gender_out = self.gender_classifier(features)
#         age_out = self.age_classifier(features)
        
#         return gender_out, age_out

# class SimpleFaceAgeGenderModel(pl.LightningModule):
#     def __init__(self, num_age_bins=101, lr=1e-3):
#         super().__init__()
#         self.save_hyperparameters()
        
#         self.cnn = SimpleCNN(num_age_bins)
        
#         self.gender_loss_fn = nn.CrossEntropyLoss()
#         self.age_loss_fn = nn.KLDivLoss(reduction='batchmean')
        
#         self.train_gender_acc = torchmetrics.classification.Accuracy(task="multiclass", num_classes=2)
#         self.val_gender_acc = torchmetrics.classification.Accuracy(task="multiclass", num_classes=2)
        
#         self.age_bins = torch.arange(0, num_age_bins).float()

#     def forward(self, x):
#         return self.cnn(x)

#     def configure_optimizers(self):
#         optimizer = torch.optim.AdamW(
#             self.parameters(), 
#             lr=self.hparams.lr,
#             weight_decay=1e-4,
#         )
        
#         scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(
#             optimizer, 
#             T_max=EPOCHS
#         )
        
#         return [optimizer], [scheduler]

#     def _shared_step(self, batch, stage):
#         imgs, genders, age_targets = batch
#         gender_logits, age_logits = self(imgs)
        
#         gender_loss = self.gender_loss_fn(gender_logits, genders)
#         age_loss = self.age_loss_fn(F.log_softmax(age_logits, dim=1), age_targets)
#         total_loss = gender_loss + age_loss
        
#         # Gender metrics
#         if stage == 'train':
#             self.train_gender_acc(gender_logits, genders)
#             gender_acc = self.train_gender_acc.compute()
#         else:
#             self.val_gender_acc(gender_logits, genders)
#             gender_acc = self.val_gender_acc.compute()
        
#         # Age metrics
#         age_probs = F.softmax(age_logits, dim=1)
#         pred_ages = (age_probs * self.age_bins.to(age_probs.device)).sum(dim=1)
#         true_ages = (age_targets * self.age_bins.to(age_targets.device)).sum(dim=1)
#         age_rmse = torch.sqrt(F.mse_loss(pred_ages, true_ages))
#         age_mae = F.l1_loss(pred_ages, true_ages)
        
#         # Log metrics to PyTorch Lightning
#         self.log(f"{stage}_loss", total_loss, prog_bar=True)
#         self.log(f"{stage}_gender_loss", gender_loss, prog_bar=True)
#         self.log(f"{stage}_age_loss", age_loss, prog_bar=True)
#         self.log(f"{stage}_gender_acc", gender_acc, prog_bar=True)
#         self.log(f"{stage}_age_rmse", age_rmse, prog_bar=True)
#         self.log(f"{stage}_age_mae", age_mae, prog_bar=True)
        
#         return total_loss

#     def training_step(self, batch, batch_idx):
#         return self._shared_step(batch, "train")

#     def validation_step(self, batch, batch_idx):
#         self._shared_step(batch, "val")

In [22]:
# # Cell 4: Data Pipeline

# class EnhancedFaceDataset(Dataset):
#     def __init__(self, df, root_dir, transforms=None, train=True):
#         self.df = df.reset_index(drop=True)
#         self.root = Path(root_dir)
#         self.transforms = transforms
#         self.train = train
        
#         if train and 'age' in df.columns:
#             self.soft_labels = self._create_soft_labels(df['age'].values)
#         else:
#             self.soft_labels = None

#     def _create_soft_labels(self, ages, sigma=1.5):
#         age_bins = np.arange(0, NUM_AGE_BINS)
#         soft_labels = []
#         for age in ages:
#             probs = np.exp(-0.5 * ((age_bins - age) / sigma) ** 2)
#             soft_labels.append((probs / probs.sum()).astype(np.float32))
#         return np.vstack(soft_labels)

#     def __len__(self):
#         return len(self.df)

#     def __getitem__(self, idx):
#         row = self.df.iloc[idx]
#         img_path = self.root / row['full_path']
        
#         try:
#             img = Image.open(img_path).convert("RGB")
#         except:
#             img = Image.new('RGB', (IMG_SIZE, IMG_SIZE), color='black')
        
#         if self.transforms:
#             img = self.transforms(img)
            
#         if self.train:
#             gender = torch.tensor(row['gender'], dtype=torch.long)
#             age_target = torch.tensor(self.soft_labels[idx], dtype=torch.float32)
#             return img, gender, age_target
#         else:
#             return img, row['id']

# def get_enhanced_transforms():
#     train_transforms = transforms.Compose([
#         transforms.Resize((IMG_SIZE, IMG_SIZE)),
#         transforms.RandomHorizontalFlip(p=0.5),
#         transforms.ColorJitter(0.2, 0.2, 0.2, 0.1),
#         transforms.RandomRotation(5),
#         transforms.ToTensor(),
#         transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
#     ])
    
#     val_transforms = transforms.Compose([
#         transforms.Resize((IMG_SIZE, IMG_SIZE)),
#         transforms.ToTensor(),
#         transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
#     ])
    
#     return train_transforms, val_transforms

# def get_enhanced_dataloaders(batch_size=BATCH_SIZE):
#     df = pd.read_csv(TRAIN_CSV)
    
#     df['age_bucket'] = pd.cut(df['age'], bins=10, labels=False)
#     df['stratify_label'] = df['age_bucket'].astype(str) + "_" + df['gender'].astype(str)
    
#     train_df, val_df = train_test_split(
#         df, test_size=0.15, random_state=SEED,
#         stratify=df['stratify_label']
#     )
    
#     train_transforms, val_transforms = get_enhanced_transforms()
    
#     train_ds = EnhancedFaceDataset(train_df, ROOT_DIR, train_transforms, train=True)
#     val_ds = EnhancedFaceDataset(val_df, ROOT_DIR, val_transforms, train=True)
    
#     train_loader = DataLoader(train_ds, batch_size=batch_size, shuffle=True, num_workers=2)
#     val_loader = DataLoader(val_ds, batch_size=batch_size, shuffle=False, num_workers=2)
    
#     # Log dataset statistics to Track.io
#     trackio.log({
#         "training_samples": len(train_ds),
#         "validation_samples": len(val_ds),
#         "total_samples": len(df)
#     })
    
#     print(f"📊 Training samples: {len(train_ds)}")
#     print(f"📊 Validation samples: {len(val_ds)}")
    
#     return train_loader, val_loader

In [23]:
# # Cell 5: Training Setup & Kaggle Hub Utilities

# from pytorch_lightning.callbacks import ModelCheckpoint, EarlyStopping, Callback

# class TrackIOCallback(Callback):
#     def on_train_start(self, trainer, pl_module):
#         trackio.log({"status": "training_started"})
        
#     def on_train_end(self, trainer, pl_module):
#         trackio.log({"status": "training_completed"})
        
#     def on_epoch_end(self, trainer, pl_module):
#         # Log epoch metrics from trainer
#         metrics = {
#             "epoch": trainer.current_epoch,
#             "train_loss": trainer.callback_metrics.get('train_loss', torch.tensor(0.0)).item(),
#             "val_loss": trainer.callback_metrics.get('val_loss', torch.tensor(0.0)).item(),
#             "val_gender_acc": trainer.callback_metrics.get('val_gender_acc', torch.tensor(0.0)).item(),
#             "val_age_mae": trainer.callback_metrics.get('val_age_mae', torch.tensor(0.0)).item(),
#         }
#         trackio.log(metrics)

# def save_model_to_kaggle_hub(model, model_name="face-age-gender-Efficientnet-B4-model"):
#     """Save model to Kaggle Hub"""
#     try:
#         # Create model directory
#         model_dir = Path("/kaggle/working/model_artifacts")
#         model_dir.mkdir(exist_ok=True)
        
#         # Save model weights
#         model_path = model_dir / "model_weights.pth"
#         torch.save(model.state_dict(), model_path)
        
#         # Save model architecture info
#         model_info = {
#             "model_name": model_name,
#             "backbone": "tf_efficientnet_b4_ns",
#             "img_size": IMG_SIZE,
#             "num_age_bins": NUM_AGE_BINS,
#             "timestamp": datetime.now().isoformat()
#         }
        
#         with open(model_dir / "model_info.json", "w") as f:
#             json.dump(model_info, f, indent=2)
        
#         print("✅ Model saved locally for Kaggle Hub")
#         return model_dir
        
#     except Exception as e:
#         print(f"❌ Failed to save model: {e}")
#         return None

# def load_model_from_kaggle_hub(model_identifier, model_class):
#     """Load model from Kaggle Hub"""
#     try:
#         print(f"📥 Downloading model from Kaggle Hub: {model_identifier}")
        
#         # Download model using kagglehub
#         model_path = kagglehub.model_download(model_identifier)
#         print(f"✅ Model downloaded to: {model_path}")
        
#         # Load model weights
#         weights_path = Path(model_path) / "model_weights.pth"
#         model_info_path = Path(model_path) / "model_info.json"
        
#         if weights_path.exists():
#             # Initialize model
#             model = model_class()
#             model.load_state_dict(torch.load(weights_path, map_location=DEVICE))
#             model.to(DEVICE)
#             model.eval()
            
#             # Load model info
#             if model_info_path.exists():
#                 with open(model_info_path, 'r') as f:
#                     model_info = json.load(f)
#                 print(f"📋 Model info: {model_info}")
            
#             print("✅ Model loaded successfully from Kaggle Hub")
#             return model
#         else:
#             print("❌ Model weights not found")
#             return None
            
#     except Exception as e:
#         print(f"❌ Failed to load model from Kaggle Hub: {e}")
#         return None

In [24]:
# # Cell 6: Training Execution

# def train_fixed_model():
#     print("🚀 Starting Fixed EfficientNet-B4 Training...")
    
#     # Log training start
#     trackio.log({"training_status": "starting"})
    
#     # Get data
#     train_loader, val_loader = get_enhanced_dataloaders()
    
#     # Initialize model
#     model = FixedFaceAgeGenderModel(lr=2e-4)
    
#     # Test forward pass
#     print("🧪 Testing forward pass...")
#     try:
#         dummy_input = torch.randn(2, 3, IMG_SIZE, IMG_SIZE)
#         gender_out, age_out = model(dummy_input)
#         print(f"✅ Forward pass successful!")
        
#         # Log model architecture info
#         trackio.log({
#             "model_backbone": "tf_efficientnet_b4_ns",
#             "feature_dim": model.backbone.num_features,
#             "gender_output_dim": gender_out.shape[1],
#             "age_output_dim": age_out.shape[1]
#         })
        
#     except Exception as e:
#         print(f"❌ Forward pass failed: {e}")
#         trackio.log({"error": f"Forward pass failed: {e}"})
#         return None, None, None
    
#     # Callbacks
#     checkpoint_callback = ModelCheckpoint(
#         monitor='val_gender_acc',
#         dirpath='/kaggle/working/fixed_face_model',
#         filename='fixed-best-{epoch:02d}-{val_gender_acc:.3f}',
#         save_top_k=1,
#         mode='max'
#     )
    
#     early_stop_callback = EarlyStopping(
#         monitor='val_loss',
#         min_delta=0.001,
#         patience=8,
#         mode='min'
#     )
    
#     track_callback = TrackIOCallback()
    
#     # Trainer
#     trainer = pl.Trainer(
#         max_epochs=EPOCHS,
#         accelerator="auto",
#         devices="auto",
#         callbacks=[checkpoint_callback, early_stop_callback, track_callback],
#         log_every_n_steps=10,
#     )
    
#     print("📈 Starting training...")
#     trackio.log({"training_status": "in_progress"})
    
#     trainer.fit(model, train_loader, val_loader)
    
#     # Save model to Kaggle Hub format
#     model_dir = save_model_to_kaggle_hub(model)
    
#     # Log final metrics
#     final_metrics = {
#         "final_train_loss": trainer.callback_metrics.get('train_loss', torch.tensor(0.0)).item(),
#         "final_val_loss": trainer.callback_metrics.get('val_loss', torch.tensor(0.0)).item(),
#         "final_val_gender_acc": trainer.callback_metrics.get('val_gender_acc', torch.tensor(0.0)).item(),
#         "final_val_age_mae": trainer.callback_metrics.get('val_age_mae', torch.tensor(0.0)).item(),
#         "training_status": "completed"
#     }
#     trackio.log(final_metrics)
    
#     print("✅ Fixed training completed!")
#     return trainer, model, model_dir

# # Run training
# trainer, model, model_dir = train_fixed_model()

In [25]:
# # Cell 6b: Simple CNN Training

# def train_simple_cnn_model():
#     print("🚀 Starting Simple CNN Training...")
    
#     # Get data
#     train_loader, val_loader = get_enhanced_dataloaders()
    
#     # Initialize simple CNN model
#     model = SimpleFaceAgeGenderModel(lr=1e-3)
    
#     # Test forward pass
#     print("🧪 Testing forward pass...")
#     try:
#         dummy_input = torch.randn(2, 3, IMG_SIZE, IMG_SIZE)
#         gender_out, age_out = model(dummy_input)
#         print(f"✅ Forward pass successful!")
#         print(f"   Gender output shape: {gender_out.shape}")
#         print(f"   Age output shape: {age_out.shape}")
        
#         # Log model architecture info
#         trackio.log({
#             "model_type": "simple_cnn",
#             "total_parameters": sum(p.numel() for p in model.parameters()),
#             "trainable_parameters": sum(p.numel() for p in model.parameters() if p.requires_grad),
#         })
        
#     except Exception as e:
#         print(f"❌ Forward pass failed: {e}")
#         trackio.log({"error": f"Forward pass failed: {e}"})
#         return None, None, None
    
#     # Callbacks
#     checkpoint_callback = ModelCheckpoint(
#         monitor='val_gender_acc',
#         dirpath='/kaggle/working/simple_cnn_model',
#         filename='simple-cnn-best-{epoch:02d}-{val_gender_acc:.3f}',
#         save_top_k=1,
#         mode='max'
#     )
    
#     early_stop_callback = EarlyStopping(
#         monitor='val_loss',
#         min_delta=0.001,
#         patience=5,
#         mode='min'
#     )
    
#     track_callback = TrackIOCallback()
    
#     # Trainer
#     trainer = pl.Trainer(
#         max_epochs=EPOCHS,
#         accelerator="auto",
#         devices="auto",
#         callbacks=[checkpoint_callback, early_stop_callback, track_callback],
#         log_every_n_steps=10,
#     )
    
#     print("📈 Starting training...")
    
#     trainer.fit(model, train_loader, val_loader)
    
#     # Save model to Kaggle Hub format
#     model_dir = save_model_to_kaggle_hub(model, "simple-cnn-face-model")
    
#     print("✅ Simple CNN training completed!")
#     return trainer, model, model_dir

# # Run training with Simple CNN (optional - you can run this separately)
# # simple_trainer, simple_model, simple_model_dir = train_simple_cnn_model()

In [26]:
# # Cell 7: Test Dataset & Inference Setup

# class TestFaceDataset(Dataset):
#     def __init__(self, test_csv, root_dir, transforms=None):
#         self.df = pd.read_csv(test_csv)
#         self.root = Path(root_dir)
#         self.transforms = transforms

#     def __len__(self):
#         return len(self.df)

#     def __getitem__(self, idx):
#         row = self.df.iloc[idx]
#         img_path = self.root / row['full_path']
#         try:
#             img = Image.open(img_path).convert("RGB")
#         except:
#             img = Image.new('RGB', (IMG_SIZE, IMG_SIZE), color='black')
#         if self.transforms:
#             img = self.transforms(img)
#         return img, row['id']

In [27]:
# # Cell 8: Enhanced Inference Function

# def enhanced_predict(use_kaggle_hub=False, model_identifier=None):
#     """Make predictions with option to load from Kaggle Hub"""
    
#     trackio.log({"inference_status": "starting"})
    
#     if use_kaggle_hub and model_identifier:
#         print("🔄 Loading model from Kaggle Hub...")
#         model = load_model_from_kaggle_hub(model_identifier, FixedFaceAgeGenderModel)
#         if model is None:
#             print("❌ Failed to load from Kaggle Hub, using local model")
#             if trainer is None:
#                 print("❌ No local model available")
#                 return None
#             checkpoint_path = trainer.checkpoint_callback.best_model_path
#             model = FixedFaceAgeGenderModel.load_from_checkpoint(checkpoint_path)
#     else:
#         if trainer is None:
#             print("❌ Training failed, cannot make predictions")
#             return None
            
#         checkpoint_path = trainer.checkpoint_callback.best_model_path
#         print(f"📁 Loading best checkpoint: {checkpoint_path}")
#         model = FixedFaceAgeGenderModel.load_from_checkpoint(checkpoint_path)
    
#     model.to(DEVICE)
#     model.eval()
    
#     # Prepare test data
#     _, test_transforms = get_enhanced_transforms()
#     test_ds = TestFaceDataset(TEST_CSV, ROOT_DIR, test_transforms)
#     test_loader = DataLoader(test_ds, batch_size=BATCH_SIZE, shuffle=False, num_workers=2)
    
#     all_ids = []
#     all_genders = []
#     all_ages = []
    
#     print("🔮 Making predictions...")
#     trackio.log({"inference_status": "in_progress"})
    
#     with torch.no_grad():
#         for batch in tqdm(test_loader, desc="Predicting"):
#             imgs, ids = batch
#             imgs = imgs.to(DEVICE)
            
#             gender_logits, age_logits = model(imgs)
            
#             gender_preds = torch.argmax(gender_logits, dim=1).cpu().numpy()
#             age_probs = F.softmax(age_logits, dim=1).cpu().numpy()
#             age_preds = (age_probs * np.arange(0, NUM_AGE_BINS)).sum(axis=1)
            
#             if isinstance(ids, torch.Tensor):
#                 ids = ids.cpu().numpy().tolist()
            
#             all_ids.extend(ids)
#             all_genders.extend(gender_preds)
#             all_ages.extend(age_preds)
    
#     # Create submission
#     submission = pd.DataFrame({
#         'id': all_ids,
#         'age': np.clip(all_ages, 0, 100).round().astype(int),
#         'gender': all_genders
#     })
    
#     # Log inference statistics
#     inference_stats = {
#         "test_samples_processed": len(submission),
#         "predicted_age_mean": submission['age'].mean(),
#         "predicted_age_std": submission['age'].std(),
#         "male_predictions": (submission['gender'] == 0).sum(),
#         "female_predictions": (submission['gender'] == 1).sum(),
#         "inference_status": "completed"
#     }
#     trackio.log(inference_stats)
    
#     return submission

In [28]:
# # Cell 8b: Simple CNN Inference Function

# def simple_cnn_predict(use_kaggle_hub=False, model_identifier=None):
#     """Make predictions with Simple CNN model"""
    
#     if use_kaggle_hub and model_identifier:
#         print("🔄 Loading Simple CNN model from Kaggle Hub...")
#         model = load_model_from_kaggle_hub(model_identifier, SimpleFaceAgeGenderModel)
#         if model is None:
#             print("❌ Failed to load from Kaggle Hub")
#             return None
#     else:
#         # You need to train the simple CNN first or load from checkpoint
#         print("❌ No Simple CNN model available. Please train first or provide Kaggle Hub identifier.")
#         return None
    
#     model.to(DEVICE)
#     model.eval()
    
#     # Prepare test data
#     _, test_transforms = get_enhanced_transforms()
#     test_ds = TestFaceDataset(TEST_CSV, ROOT_DIR, test_transforms)
#     test_loader = DataLoader(test_ds, batch_size=BATCH_SIZE, shuffle=False, num_workers=2)
    
#     all_ids = []
#     all_genders = []
#     all_ages = []
    
#     print("🔮 Making predictions with Simple CNN...")
    
#     with torch.no_grad():
#         for batch in tqdm(test_loader, desc="Predicting"):
#             imgs, ids = batch
#             imgs = imgs.to(DEVICE)
            
#             gender_logits, age_logits = model(imgs)
            
#             gender_preds = torch.argmax(gender_logits, dim=1).cpu().numpy()
#             age_probs = F.softmax(age_logits, dim=1).cpu().numpy()
#             age_preds = (age_probs * np.arange(0, NUM_AGE_BINS)).sum(axis=1)
            
#             if isinstance(ids, torch.Tensor):
#                 ids = ids.cpu().numpy().tolist()
            
#             all_ids.extend(ids)
#             all_genders.extend(gender_preds)
#             all_ages.extend(age_preds)
    
#     # Create submission
#     submission = pd.DataFrame({
#         'id': all_ids,
#         'age': np.clip(all_ages, 0, 100).round().astype(int),
#         'gender': all_genders
#     })
    
#     return submission



In [29]:
# # Simple CNN for inference
# simple_submission = simple_cnn_predict()
# if simple_submission is not None:
#     simple_submission.to_csv('simple_cnn_submission.csv', index=False)
#     print("✅ Simple CNN submission saved!")

In [30]:
# # Final Inference & Results

# # Run enhanced inference
# if trainer is not None:
#     print("🚀 Starting enhanced inference...")
    
#     # Option 1: Use local model (default)
#     enhanced_submission = enhanced_predict(use_kaggle_hub=False)
    
#     # Option 2: Uncomment to use Kaggle Hub model
#     # enhanced_submission = enhanced_predict(
#     #     use_kaggle_hub=True, 
#     #     model_identifier="your-username/face-age-gender-model"
#     # )
    
#     if enhanced_submission is not None:
#         # Save results
#         enhanced_submission.to_csv('submission.csv', index=False)
        
#         print("\n✅ Enhanced inference completed!")
#         print(f"📊 Processed {len(enhanced_submission)} test samples")
#         print("\n📋 Enhanced Submission preview:")
#         print(enhanced_submission.head(10))
        
#         # Show statistics
#         print(f"\n📈 Enhanced Submission Statistics:")
#         print(f"Age range: {enhanced_submission['age'].min()} - {enhanced_submission['age'].max()}")
#         print(f"Mean age: {enhanced_submission['age'].mean():.1f}")
#         print(f"Gender distribution:")
#         print(f"  Male (0): {(enhanced_submission['gender'] == 0).sum()}")
#         print(f"  Female (1): {(enhanced_submission['gender'] == 1).sum()}")
        
#         print(f"\n📁 Enhanced submission saved: enhanced_submission.csv")
        
#         # Convert numpy types to native Python types for Track.io logging
#         trackio.log({
#             "inference_status": "success",
#             "processed_samples": int(len(enhanced_submission)),  # Convert to native int
#             "age_min": int(enhanced_submission['age'].min()),    # Convert to native int
#             "age_max": int(enhanced_submission['age'].max()),    # Convert to native int
#             "age_mean": float(enhanced_submission['age'].mean()), # Convert to native float
#             "male_count": int((enhanced_submission['gender'] == 0).sum()),  # Convert to native int
#             "female_count": int((enhanced_submission['gender'] == 1).sum()) # Convert to native int
#         })
        
# else:
#     print("❌ Training failed, creating fallback submission...")
    
#     # Fallback
#     test_df = pd.read_csv(TEST_CSV)
#     enhanced_submission = pd.DataFrame({
#         'id': test_df['id'].values,
#         'age': np.random.randint(20, 60, len(test_df)),
#         'gender': np.random.randint(0, 2, len(test_df))
#     })
#     enhanced_submission.to_csv('submission.csv', index=False)
#     print("✅ Fallback submission created")
    
#     # Log fallback with native types
#     trackio.log({
#         "inference_status": "fallback",
#         "processed_samples": int(len(enhanced_submission))
#     })

# # Finish Track.io run - add error handling
# try:
#     trackio.finish()
#     print("✅ Track.io logging completed successfully!")
# except Exception as e:
#     print(f"⚠️ Track.io finish had an issue (non-critical): {e}")

# print("🎯 Pipeline completed with Track.io logging and Kaggle Hub integration!")